In [1]:
%%capture
import re
import glob
from os import path
from datetime import datetime
import numpy as np

import geopandas as gpd
import fiona
import rioxarray as rxr
import xarray as xr
from shapely.geometry import mapping
from geocube.api.core import make_geocube
import cv2

from skimage.io import imread

import matplotlib.pyplot as plt


In [2]:
MASKS_PATH = 'data\mask_per_year.gpkg'
IMAGES_PATH = 'data\images'

grid = gpd.read_file('data\grid.gpkg')
years = fiona.listlayers(MASKS_PATH)[1:] # list layers/years

# read the masks 
mask_per_year = {key: gpd.read_file(MASKS_PATH, layer=key) for key in years}

# add a 1 column
for value in mask_per_year.values():
    value['value'] = 1

# CRS assertion 
for year in years:
    assert mask_per_year[year].crs == grid.crs, 'CRS must match'
assert grid.is_valid.all(), 'Invalid geometry'

# clip by grid extent 
mask_per_year = {key: values.clip(grid) for key, values in mask_per_year.items()}

In [3]:
paths = glob.glob(IMAGES_PATH +'/*')
paths

['data\\images\\1942.tif',
 'data\\images\\1962.tif',
 'data\\images\\1971.tif',
 'data\\images\\1984.tif',
 'data\\images\\1992.tif',
 'data\\images\\2002.tif']

In [4]:
imgs_by_year = []
SIZES = []

for imgs in paths:
    # get the year name in the file path 
    year = re.findall(r"[\w+']+", imgs)[-2]

    xr_imgs = rxr.open_rasterio(imgs).squeeze().astype(np.uint8)
    

    # set time attributes 
    xr_imgs.attrs['year'] = year

    # store image sizes (may be used in a later stage to resample the raster)
    SIZES.append(xr_imgs.shape)
    SIZES.sort()

    # Save clipped raster
    if not path.exists(f"data/clipped_images/{year}.tif"):
        print('clipping image from', year)
        # clip by grid extent 
        xr_imgs_clipped = xr_imgs.rio.clip(grid.geometry.apply(mapping), grid.crs)

        # save geotif
        xr_imgs_clipped.rio.to_raster(f'data/clipped_images/{year}.tif')

    else:
        print(f'{year} is already clipped!')

1942 is already clipped!
1962 is already clipped!
1971 is already clipped!
1984 is already clipped!
1992 is already clipped!
2002 is already clipped!


In [6]:
def rasterize_vect(df):
    return make_geocube(vector_data=df,
    measurements=["value"],
    resolution=(0.75,-0.75),
    fill=0).astype(np.uint8)
    
for year in years:
    if not path.exists(f'data/mask/{year}.tif'):
        dataset = rasterize_vect(mask_per_year[year])
        out_grid_2 = dataset.to_array()
        out_grid_2.rio.to_raster(f'data/mask/{year}.tif')